<a href="https://www.kaggle.com/code/ramsesmdlc/mnist-classification?scriptVersionId=139569413" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# __0. Libraries__

In [2]:
#Import Libraries 

import numpy as np # linear algebra
import pandas as pd # data processing

import matplotlib as mpl # data visualization
import matplotlib.pyplot as plt # data visualization

# __1. Important__

Type of machine learning system to build:

1. Supervised Learning: __Classification__
2. Batch Learning (also called "offline learning")
3. Model-based learning

Goal:
1. __It is to take an image of a handwritten single digit, and determine what that digit is.__

2. __Metric__: This competition is evaluated on the categorization accuracy of your predictions (the percentage of images you get correct).

# __2. Loading the Data - Training Dataset__

In [9]:
training_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
print(training_data.head(3))

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  

[3 rows x 785 columns]


In [6]:
training_data.keys()

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [8]:
print(type(training_data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
def EDA(training_dataX):  
  # Print the "shape" of the dataframe
  print("\n" +'\033[1m','\033[94m',"Shape of the dataframe - some metric variables:",'\033[0m', training_dataX.shape, "\n")
    
  # Print the "head" of the dataframe
  print('\033[1m','\033[94m',"Head of the dataframe - some metric variables:",'\033[0m', "\n")
  print(training_dataX.head())
  
  # Print the "general information" of the dataframe
  print("\n", '\033[1m','\033[94m',"Information of the dataframe - some metric variables:",'\033[0m', "\n")
  training_dataX.info()
 
  # Print the "number and percentage" of missing values per column" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m', "Number and percentage of missing values per column of some metric variables of the dataframe:", '\033[0m' + "\n")
  missing = training_dataX.isnull().sum()
  percent = missing / training_dataX.shape[0] * 100
  print(pd.concat([missing, percent], axis=1, keys=["Missing", "Percent"]))

  # Print the "summary statistics" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m', "Summary statistics of some metric variables of the dataframe:", '\033[0m' + "\n")
  print(training_dataX.describe())  
    
  # Plot "histograms" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m',"Histograms of some metric variables of the dataframe:", '\033[0m' + "\n")
  training_dataX.hist(bins= 50, figsize=(20,20),color= "Crimson")
  plt.show()  
    
  # Plot "boxplots" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m',"Boxplots of some metric variables of the dataframe:", '\033[0m' + "\n")
  training_dataX.boxplot(figsize=(20,20), rot=90, color= "Crimson")
  plt.show()
  
  # Print the "Spearman Correlation Coefficient" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m', "Spearman Correlation Coefficient of some metric variables of the dataframe:", '\033[0m' + "\n")
  print(training_dataX.corr(method="spearman"))    
    
  # Plot "heatmap" of the "Spearman Correlation Coefficient" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m', "Spearman Correlation matrix of some metric variables of the dataframe:", '\033[0m' + "\n")
  corr = training_dataX.corr(method="spearman")
  sns.set(rc={"figure.figsize":(15, 15)}) #width, height
  sns.heatmap(corr, annot = True, cmap="Spectral")
  plt.show()
    
# Call the function
EDA(training_dataX)